<a href="https://colab.research.google.com/github/nurshatfateh/WebScraping-NaturalLanguageProcessing/blob/main/Final_NLP_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/nurshatfateh/WebScraping-NaturalLanguageProcessing/main/newsdata.csv'
collection = pd.read_csv(url,encoding = 'unicode_escape')
collection


,date,url
0,8/31/2020 22:41,https://www.dhakatribune.com/bangladesh/nation...
1,8/31/2020 22:41,http://m.thedailynewnation.com/news/261789/sal...
2,8/31/2020 22:41,https://bdnews24.com/bangladesh/2020/08/30/dha...
3,8/31/2020 22:46,https://www.thedailystar.net/opinion/news/pass...
4,8/31/2020 22:47,https://www.dhakatribune.com/bangladesh/nation...
...,...,...
6622,"Jan 25, 2023, 04:54:46",https://www.latestly.com/socially/india/news/s...
6623,"Jan 25, 2023, 19:54:15",https://www.dhakatribune.com/op-ed/2023/01/26/...
6624,"Jan 25, 2023, 19:54:15",https://bdnews24.com/bangladesh/pvxt6pnv8s
6625,"Jan 25, 2023, 21:01:41",https://bdnews24.com/world/south-east-asia/2wu...


In [ ]:
collection['url'][0]

'https://www.dhakatribune.com/bangladesh/nation/2020/08/30/teenage-cleaner-killed-in-dhaka-road-crash'

In [ ]:
url = 'https://raw.githubusercontent.com/nurshatfateh/WebScraping-NaturalLanguageProcessing/main/dictionary.csv'
df = pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6
0,1,1,comilla,কুমিল্লা,23.4682747,91.178814,www.comilla.gov.bd
1,2,1,feni,ফেনী,23.023231,91.384084,www.feni.gov.bd
2,3,1,brahmanbaria,ব্রাহ্মণবাড়িয়া,23.9570904,91.111929,www.brahmanbaria.gov.bd
3,4,1,rangamati,রাঙ্গামাটি,22.65561018,92.175411,www.rangamati.gov.bd
4,5,1,noakhali,নোয়াখালী,22.869563,91.099398,www.noakhali.gov.bd


In [ ]:
pip install newspaper3k


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 70.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=b39d26910606a0c03690c2e0f6bf8cda1ed3faac03597150ced93dc01dfe7b55
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=bfd3b26831ef7307f38f1150823b7de38b2ac79cd60a216fc30b

In [ ]:
from newspaper import Article
import spacy
from spacy.matcher import Matcher

In [ ]:
def extract_article_from_url(url):
  a = Article(url)
  a.download()
  a.parse()
  return a.text, a.publish_date

In [ ]:
def sentence_giver(doc, start, end):
  s = start
  while str(doc[s]) != '.':
    s = s - 1
    if s < 0:
      break
  e = end
  while str(doc[e]) != '.':

    e = e + 1

    if e > len(doc) - 1:
      break

  return str(doc[s+1:e])

In [ ]:
def location_finder(doc):
  locations = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"ENT_TYPE": "GPE"}]
  matcher.add("Location", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = l_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      locations.append(doc[start:end])
  return locations, sentences

def l_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LEMMA": "accident", "POS": "VERB"}]
  pattern2 = [{"LEMMA": "die", "POS": "VERB"}]
  pattern3 = [{"LEMMA": "collide", "POS": "VERB"}]
  pattern4 = [{"LEMMA": "crash", "POS": "VERB"}]
  pattern5 = [{"LEMMA": "kill", "POS": "VERB"}]
  pattern0 = [{"LOWER": "in"}]

  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) != 0:
    return 1
  else:
    matcher.add("N_Filter", [pattern2])
    matches = matcher(data)
    if len(matches) != 0:
      return 1
    else:
      matcher.add("N_Filter", [pattern3])
      matches = matcher(data)
      if len(matches) != 0:
        return 1
      else:
        matcher.add("N_Filter", [pattern4])
        matches = matcher(data)
        if len(matches) != 0:
          return 1
        else:
          matcher.add("N_Filter", [pattern5])
          matches = matcher(data)
          if len(matches) != 0:
            return 1
          else:
            matcher.add("N_Filter", [pattern0])
            matches = matcher(data)
            if len(matches) != 0:
              return 1
            else:
              return 0

In [ ]:
def number_finder(doc):
  numbers = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"LIKE_NUM": True}]
  matcher.add("Number", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = n_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      numbers.append(str(doc[start:end]))
  return numbers, sentences

In [ ]:
def n_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LEMMA": "kill", "POS": "VERB"}]
  pattern2 = [{"LEMMA": "die", "POS": "VERB"}]


  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) != 0:
    return 1
  else:
    matcher.add("N_Filter", [pattern2])
    matches = matcher(data)
    if len(matches) != 0:
      return 1
    else:
      return 0

In [ ]:
loc=[]
dea=[]
veh1=[]
veh2=[]
veh3=[]
dat=[]
for zz in range(0,6625):

  print(zz)
  link=collection['url'][zz]





  try:
    article=Article(link)
    article.download()
    article.parse()
  except:
    continue
 
  import spacy 
  nlp = spacy.load('en_core_web_sm')
  sentence = article.text
  sentence = sentence.replace("-", ".")
  doc = nlp(sentence)




  for ent in doc.ents:
     for index, row in df.iterrows():
          if ent.text.lower()==row[2]:
           ent.label_='GPE'
           break     
  #print("---------------------------------------------------START-----------------------------------------------")
  #print(article.text)


  try:
    l_places, l_s = location_finder(doc)
  #print("ACCIDENT LOCATION: ",l_places)
  #print(l_s)
  except:
    continue

  try:  
    n_people, n_s = number_finder(doc)
    death=0
  except:
    continue

  if len(n_s)!=0:
    death=n_people[0]
    #print("Victim NUMBER: ",death)
  else:
    death="One"
    #print("Victim NUMBER: ",death)
  #print(n_s)
  z=[None,None,None]
  x=0
  #print("RELATED VEHICLES: ")
  vehicle = ['truck','bus','car','bike','motorcycle','rickshaw','tempo']
  for i in vehicle:
    for j in doc.text.split():
      if i in j:
        if j!='carrying':
          #print(i)
          z.insert(x, i)
          x=x+1
          break
  #print("News Date: ",collection['date'][zz])
  date=collection['date'][zz]
 
  
  if len(l_places) != 0:
    loc.append(l_places[0])
    dea.append(death)
    veh1.append(z[0])
    veh2.append(z[1])
    veh3.append(z[2])
    dat.append(date)  
    if len(l_places)>1:
      loc.append(l_places[1])
      dea.append(death)
      veh1.append(z[0])
      veh2.append(z[1])
      veh3.append(z[2])
      dat.append(date)
  
 

 
  #print("-------------------------------------------------END-------------------------------------------------")
  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
d = {'ACCIDENT LOCATION':loc,'ACCIDENT Date':dat,'Victim NUMBER':dea,'Vehicle 1':veh1,'Vehicle 2':veh2,'Vehicle 3':veh3}
df = pd.DataFrame(d)
df = df.dropna(axis=0, subset=['Vehicle 1'])

df

,ACCIDENT LOCATION,ACCIDENT Date,Victim NUMBER,Vehicle 1,Vehicle 2,Vehicle 3
2,(Habiganj),9/4/2020 22:02,One,rickshaw,None,None
3,(Habiganj),9/4/2020 22:02,One,rickshaw,None,None
4,(Nishikanda),9/4/2020 22:02,Three,bus,rickshaw,None
5,(Boalkhali),9/4/2020 22:02,One,bus,car,tempo
6,(Munshiganj),9/4/2020 22:02,One,bus,car,tempo
7,(Paraichak),9/4/2020 22:02,Two,bus,None,None
8,(Dhamrai),9/5/2020 22:42,3,bus,None,None
9,(Bangladesh),9/5/2020 22:42,3,bus,None,None
10,(Kaliganj),9/5/2020 22:42,One,truck,bus,car
13,(Nababganj),9/5/2020 22:46,Three,bus,rickshaw,None
